In [4]:
#| default_exp meiosis

In [5]:
#| export


import jax
import jax.numpy as jnp
from jax import lax
from functools import partial
from typing import Tuple
from chewc.structs import GeneticMap

@partial(jax.jit, static_argnames=("max_crossovers",))
def _sample_chiasmata(
    key: jax.Array,
    map_length: float,
    interference_nu: float,
    max_crossovers: int = 20
) -> jax.Array:
    """
    Generates crossover positions along a chromosome using a Gamma process.
    This models chiasma interference.
    """
    # Parameters for the Gamma distribution that models the distance between chiasmata
    shape = interference_nu
    scale = 1.0 / (2.0 * interference_nu)

    def scan_body(carry, _):
        key, last_pos = carry
        key, subkey = jax.random.split(key)
        distance = jax.random.gamma(subkey, shape) * scale
        new_pos = last_pos + distance
        return (key, new_pos), new_pos

    key, initial_key = jax.random.split(key)
    # Start sampling slightly before the chromosome to capture all events
    initial_start_pos = jax.random.uniform(initial_key, minval=-10.0, maxval=0.0)
    
    _, crossover_positions = lax.scan(
        scan_body, (key, initial_start_pos), None, length=max_crossovers
    )
    
    # A crossover is valid only if it falls within the chromosome's genetic map
    valid_mask = (crossover_positions > 0) & (crossover_positions < map_length)
    # Use jnp.where to keep array size static, filling invalid spots with NaN
    return jnp.where(valid_mask, crossover_positions, jnp.nan)

@partial(jax.jit, static_argnames=("max_crossovers",))
def _create_recombinant_chromosome(
    key: jax.Array,
    parent_chr_geno: jax.Array,    # Shape (2, n_loci)
    parent_chr_ibd: jax.Array,     # Shape (2, n_loci)
    locus_positions: jax.Array,  # Shape (n_loci,)
    interference_nu: float,
    max_crossovers: int = 20
) -> Tuple[jax.Array, jax.Array]:
    """Creates a single recombinant chromosome from a parent's two haplotypes."""
    key, chiasma_key, hap_key = jax.random.split(key, 3)
    
    map_length = locus_positions[-1]
    crossover_positions = _sample_chiasmata(chiasma_key, map_length, interference_nu, max_crossovers)
    
    # Find the locus index after each valid crossover
    crossover_indices = jnp.searchsorted(locus_positions, crossover_positions)

    # Randomly choose which parental haplotype (0 or 1) to start with
    start_hap = jax.random.randint(hap_key, (), 0, 2)
    
    # This vectorized approach determines the haplotype choice for every locus at once
    locus_segments = jnp.searchsorted(crossover_indices, jnp.arange(locus_positions.shape[0]), side='right')
    haplotype_choice = (start_hap + locus_segments) % 2
    
    # Apply the same choice mask to both genotypes and IBD arrays
    gamete_geno = jnp.where(haplotype_choice == 0, parent_chr_geno[0], parent_chr_geno[1])
    gamete_ibd = jnp.where(haplotype_choice == 0, parent_chr_ibd[0], parent_chr_ibd[1])
    
    return gamete_geno, gamete_ibd

@jax.jit
def create_gamete(
    key: jax.Array,
    parent_geno: jax.Array,  # Shape: (n_chr, 2, n_loci)
    parent_ibd: jax.Array,   # Shape: (n_chr, 2, n_loci)
    genetic_map: GeneticMap,
    interference_nu: float = 4.0 # Common value for chi-squared model
) -> Tuple[jax.Array, jax.Array]:
    """
    Creates a single gamete from a parent's full genotype and IBD.
    """
    chr_keys = jax.random.split(key, num=parent_geno.shape[0])
    
    # vmap applies the single-chromosome function to all chromosomes in parallel.
    vmapped_recombine = jax.vmap(
        _create_recombinant_chromosome, in_axes=(0, 0, 0, 0, None, None)
    )
    
    gamete_geno, gamete_ibd = vmapped_recombine(
        chr_keys, parent_geno, parent_ibd, genetic_map.locus_positions, interference_nu
    )
    return gamete_geno, gamete_ibd

In [6]:
#| hide
import nbdev; nbdev.nbdev_export()